# VR ASSIGNMENT 2 - Q2b - IMT2019092

In this question I have implemented CIFAR 10-class classification using the CIFAR-10 dataset. I have implemented it using the Bag of visual words approach. I have extracted image descriptors using SIFT. Then used K-means to form clusters and extracted bag of visual words. Finally applied Tfidf transformer and tried to train using Logistic regression, SVM and KNN models.


In [1]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import pickle

In [2]:
#Extracting train images
dir_path = '../input/cifar10-pngs-in-folders/cifar10/train'
categories = os.listdir(dir_path)
categories

In [3]:
label = -1
train_images = []
trainLabels = []
for category in categories:
    label+=1
    path = dir_path +'/'+category
    for img in os.listdir(path):
        train_images.append(cv2.imread(path + "/" + img))
        trainLabels.append(label)

In [4]:
dir_path = '../input/cifar10-pngs-in-folders/cifar10/test'
categories = os.listdir(dir_path)
categories

In [5]:
#Extracting test images
label = -1
test_images = []
testLabels = []
for category in categories:
    label+=1
    path = dir_path +'/'+category
    for img in os.listdir(path):
        test_images.append(cv2.imread(path + "/" + img))
        testLabels.append(label)

In [8]:
#Extracting decriptors using SIFT
all_descriptors = np.asarray([])
extractor = cv2.SIFT_create()
no_indices = []
for idx,img in enumerate(train_images):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kps, descriptors = extractor.detectAndCompute(gray, None)
    if type(descriptors)==np.ndarray:
        if all_descriptors.shape[0]==0:
            all_descriptors = descriptors
        else:
            all_descriptors = np.concatenate((all_descriptors, descriptors), axis=0)
    else:
        no_indices.append(idx)

In [9]:
#Running KMeans with 50 clusters
kmeans = KMeans(n_clusters = 50)
kmeans.fit(all_descriptors)

In [12]:
#Function to extract features
def get_bovw(imgs):
    bovw = []
    extractor = cv2.SIFT_create()
    for img in imgs:
        bokps = [0]*500
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        kps, descriptors = extractor.detectAndCompute(gray, None)
        if type(descriptors) == np.ndarray:
            pred = kmeans.predict(descriptors)
            for e in pred:
                bokps[e] = bokps[e] + 1
        bovw.append(bokps)

    features = np.asarray(bovw)
    return features

In [11]:
pickle.dump(kmeans, open('kmfile_cifar.sav', 'wb'))
#vocab = pickle.load(open('kmfile_cifar.sav', 'rb'))

In [14]:
#Running tfidf transformer
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(sublinear_tf = True)
train_features = get_bovw(train_images)
tfidf_features = tfidf_transformer.fit_transform(train_features)
test_features = get_bovw(test_images)
tfidf_test_features = tfidf_transformer.transform(test_features)

## Without TFIDF

### SVM

In [15]:
model = svm.SVC(C=1,kernel='linear')
model.fit(train_features, trainLabels)

In [16]:
accuracy = model.score(test_features, testLabels)
print(accuracy)

In [17]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
plot_confusion_matrix(model, test_features, testLabels)
plt.show()

### Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train_features, trainLabels)

In [19]:
accuracy = model.score(test_features, testLabels)
print(accuracy)

In [20]:
plot_confusion_matrix(model, test_features, testLabels)
plt.show()

### KNN

In [21]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
model.fit(train_features, trainLabels)

In [22]:
accuracy = model.score(test_features, testLabels)
print(accuracy)

In [23]:
plot_confusion_matrix(model, test_features, testLabels)
plt.show()

## With TFIDF

### SVM

In [25]:
model = svm.SVC(C=1,kernel='linear')
model.fit(tfidf_features, trainLabels)

In [26]:
accuracy = model.score(tfidf_test_features, testLabels)
print(accuracy)

In [28]:
plot_confusion_matrix(model, tfidf_test_features, testLabels)
plt.show()

### Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(tfidf_features, trainLabels)

In [30]:
accuracy = model.score(tfidf_test_features, testLabels)
print(accuracy)

In [31]:
plot_confusion_matrix(model, tfidf_test_features, testLabels)
plt.show()

### KNN

In [32]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
model.fit(train_features, trainLabels)

In [33]:
accuracy = model.score(test_features, testLabels)
print(accuracy)

In [34]:
plot_confusion_matrix(model, test_features, testLabels)
plt.show()